# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soap to download the content of the front page.

In [1]:
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace,split
from composable import from_toolz as tlz
from composable import pipeable
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling
import re

In [21]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=0')
yelp_restaurant = BeautifulSoup(r.content, "html.parser")

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [3]:

restaurant_name = (yelp_restaurant
>>find_all('a')
                  )
restaurant_name

[<a class="link__09f24__1kwXV logo-link__09f24__3oxr_ link-color--blue-dark__09f24__2DRa0 link-size--default__09f24__3xWLF" href="/" name="" rel="" role="link" target="">Yelp</a>,
 <a class="button__09f24__1hAjX secondary__09f24__3mm8J find-near-button__09f24__UO8nW" data-testid="responsive-suggest-submit" href="#" style="--mousedown-x:0px;--mousedown-y:0px;--button-width:0px"><div class="button-content__09f24__2SF6G border-color--default__09f24__R1nRO"><span class="text__09f24__2tZKC button-content-text__09f24__3OonI text-color--inherit__09f24__1jgBv text-align--center__09f24__3NO89 text-weight--semibold__09f24__MTlNc text-size--large__09f24__3-9KJ text--truncated__09f24__2NCmr">Cancel</span></div></a>,
 <a class="header-link__09f24__3OkYO" href="/advertise/consumer_header_redirect"><div class="padding-t1__09f24__2GVpG padding-r1__09f24__23Vay padding-b1__09f24__1d8yO padding-l1__09f24__aqYTU border-color--default__09f24__R1nRO"><div class="notification-wrapper__09f24__3YKGg display--

In [4]:

not_empty = re.compile(r'.+')
restaurant_name = (yelp_restaurant
>>find_all('a',attrs = {'name': not_empty})
>> map(tlz.get('name'))
)
restaurant_name

['Pappi’s Taqueria y Mas',
 'The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'Piggy’s Restaurant',
 'The Crow',
 'Digger’s Sting Restaurant']

In [5]:
len(restaurant_name)

11

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [6]:
is_ranking = re.compile(r'black-regular')
restaurant_ranking = (yelp_restaurant
>>find_all('span', attrs = {'class':is_ranking})
>>map(get_text)
)
restaurant_ranking

['Curbside Pickup',
 'Open Now\xa0-:-- am',
 'Open Now\xa0-:-- am',
 'Yelp Delivery',
 'Good for Kids',
 'Good for Groups',
 'Has TV',
 'Full Bar',
 "Bird's-eye View",
 'Driving (5 mi.)',
 'Biking (2 mi.)',
 'Walking (1 mi.)',
 'Within 4 blocks',
 'Sort:',
 'Recommended',
 'Pappi’s Taqueria y Mas',
 '1.\xa0The Waterfront Restaurant & Tavern',
 '2.\xa0Restore Public House',
 '3.\xa0Buzzard Billy’s',
 '4.\xa0Lovechild Restaurant',
 '5.\xa0The Charmant',
 '6.\xa0The Freighthouse Restaurant',
 '7.\xa0Howie’s on La Crosse',
 '8.\xa0Piggy’s Restaurant',
 '9.\xa0The Crow',
 '10.\xa0Digger’s Sting Restaurant']

In [7]:
rank_filter = re.compile(r'\d{1,2}')
rankings = [line for line in restaurant_ranking if rank_filter.match(line)]
rankings

['1.\xa0The Waterfront Restaurant & Tavern',
 '2.\xa0Restore Public House',
 '3.\xa0Buzzard Billy’s',
 '4.\xa0Lovechild Restaurant',
 '5.\xa0The Charmant',
 '6.\xa0The Freighthouse Restaurant',
 '7.\xa0Howie’s on La Crosse',
 '8.\xa0Piggy’s Restaurant',
 '9.\xa0The Crow',
 '10.\xa0Digger’s Sting Restaurant']

In [8]:
[line.replace('\xa0',"") for line in rankings]

['1.The Waterfront Restaurant & Tavern',
 '2.Restore Public House',
 '3.Buzzard Billy’s',
 '4.Lovechild Restaurant',
 '5.The Charmant',
 '6.The Freighthouse Restaurant',
 '7.Howie’s on La Crosse',
 '8.Piggy’s Restaurant',
 '9.The Crow',
 '10.Digger’s Sting Restaurant']

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [9]:
not_empty = re.compile(r'.+')
restaurant_name = (yelp_restaurant
>>find_all('a',attrs = {'name': not_empty})
>> map(tlz.get('name'))
)
restaurant_name

['Pappi’s Taqueria y Mas',
 'The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'Piggy’s Restaurant',
 'The Crow',
 'Digger’s Sting Restaurant']

#### Rating

In [10]:
star_rating = re.compile(r'\d+ star rating')
review_ranking = (yelp_restaurant
>>find_all('div', attrs = {'aria-label': star_rating})
>>map(tlz.get('aria-label'))

)
review_ranking

['4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '4 star rating',
 '3.5 star rating',
 '4 star rating']

#### Address

In [11]:
find_address = re.compile(r'raw_')

restaurant_address_search = (yelp_restaurant
>>find_all('span', attrs = {'class': find_address})
>>map(get_text)
)
restaurant_address_search

['Best Restaurants in La Crosse, WI',
 '83 Buol Rd',
 'Delivery',
 'Takeout',
 'Outdoor Seating',
 '328 Front St S',
 'Dine-in',
 '1810 State St',
 'Outdoor Seating',
 '222 Pearl St',
 'Takeout',
 'Dine-in',
 '300 3rd St S',
 'Takeout',
 '101 State St',
 '107 Vine St',
 'Delivery',
 'Takeout',
 'Outdoor Seating',
 '1128 La Crosse St',
 'Delivery',
 'Takeout',
 'Outdoor Seating',
 '501 Front St S',
 'Delivery',
 'Takeout',
 'Outdoor Seating',
 '100 3rd St S',
 'Delivery',
 'Takeout',
 'Outdoor Seating',
 '122 3rd St N']

In [12]:

address_filter = re.compile(r'\d{1,4}.*')
restaurant_address=[line for line in restaurant_address_search if address_filter.match(line)]
restaurant_address


['83 Buol Rd',
 '328 Front St S',
 '1810 State St',
 '222 Pearl St',
 '300 3rd St S',
 '101 State St',
 '107 Vine St',
 '1128 La Crosse St',
 '501 Front St S',
 '100 3rd St S',
 '122 3rd St N']

#### Review Count

In [13]:
find_reveiw_count = re.compile(r'reviewCount_')
restaurant_reveiw_count = (yelp_restaurant
>>find_all('span', attrs = {'class': find_reveiw_count})
>>map(get_text)
)
restaurant_reveiw_count

['2', '226', '17', '276', '109', '149', '125', '57', '131', '154', '62']

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

In [14]:
restaurant_category = (yelp_restaurant
>>find_all('div')

)
restaurant_category

[<div data-hypernova-id="da637f0e-50b6-4754-9ba7-2686296b4af8" data-hypernova-key="yelpfrontend__4470__yelpfrontend__GondolaSearch__dynamic"><style data-emotion-css="xl0fej">html,body,div,span,applet,object,iframe,h1,h2,h3,h4,h5,h6,p,blockquote,pre,a,abbr,acronym,address,big,cite,code,del,dfn,em,img,ins,kbd,q,s,samp,small,strike,strong,sub,sup,tt,var,b,u,i,center,dl,dt,dd,ol,ul,li,fieldset,form,label,legend,table,caption,tbody,tfoot,thead,tr,th,td,article,aside,canvas,details,embed,figure,figcaption,footer,header,hgroup,menu,nav,output,ruby,section,summary,time,mark,audio,video{margin:0;padding:0;border:0;font-size:100%;font:inherit;vertical-align:baseline;}article,aside,details,figcaption,figure,footer,header,hgroup,menu,nav,section{display:block;}body{line-height:1;}ol,ul{list-style:none;}blockquote,q{quotes:none;}blockquote:before,blockquote:after,q:before,q:after{content:'';content:none;}table{border-collapse:collapse;border-spacing:0;}</style><div class="alert-notice__09f24__2tSdZ

In [15]:
find_category = re.compile(r'priceCategory')
restaurant_category = (yelp_restaurant
>>find_all('div', attrs = {'class': find_category})
>>map(get_text)                  
)
restaurant_category

['Mexican, Food Trucks',
 '$$$American (New), Seafood, Steakhouses',
 'American (Traditional)',
 '$$American (Traditional), Cajun/Creole',
 '$$$American (New)',
 '$$French, Cocktail Bars',
 '$$$Seafood, Steakhouses, Desserts',
 '$$American (New), Pubs',
 '$$Steakhouses, Seafood, Sandwiches',
 '$$American (Traditional), Gastropubs',
 '$$$American (New), Steakhouses']

In [16]:
category_filter = [line.replace('$',"") for line in restaurant_category]
category_filter

['Mexican, Food Trucks',
 'American (New), Seafood, Steakhouses',
 'American (Traditional)',
 'American (Traditional), Cajun/Creole',
 'American (New)',
 'French, Cocktail Bars',
 'Seafood, Steakhouses, Desserts',
 'American (New), Pubs',
 'Steakhouses, Seafood, Sandwiches',
 'American (Traditional), Gastropubs',
 'American (New), Steakhouses']

In [17]:
category_semi = [line.replace(',',";") for line in category_filter]
category_semi

['Mexican; Food Trucks',
 'American (New); Seafood; Steakhouses',
 'American (Traditional)',
 'American (Traditional); Cajun/Creole',
 'American (New)',
 'French; Cocktail Bars',
 'Seafood; Steakhouses; Desserts',
 'American (New); Pubs',
 'Steakhouses; Seafood; Sandwiches',
 'American (Traditional); Gastropubs',
 'American (New); Steakhouses']

## <font color="red">  Problem 4 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

In [28]:
def webscraping_yelp(website):
    
    import requests
    from bs4 import BeautifulSoup
    
    s = requests.Session()
    r = s.get(website)
    yelp_restaurant = BeautifulSoup(r.content, "html.parser")
    
    
    is_ranking = re.compile(r'black-regular')
    restaurant_ranking = (yelp_restaurant
    >>find_all('span', attrs = {'class':is_ranking})
    >>map(get_text)
    )
    
    rank_filter = re.compile(r'\d{1,2}')
    rankings = [line for line in restaurant_ranking if rank_filter.match(line)]
    final_rank_name = [line.replace('\xa0'," ") for line in rankings]
    
    
    
    star_rating = re.compile(r'\d+ star rating')
    review_ranking = (yelp_restaurant
    >>find_all('div', attrs = {'aria-label': star_rating})
    >>map(tlz.get('aria-label'))
    )
    review_ranking= review_ranking[1:]
    
    find_address = re.compile(r'raw_')
    restaurant_address_search = (yelp_restaurant
    >>find_all('span', attrs = {'class': find_address})
    >>map(get_text)
    )
    address_filter = re.compile(r'\d{1,4}.*')
    restaurant_address=[line for line in restaurant_address_search if address_filter.match(line)]
    restaurant_address = restaurant_address[1:]
    
    find_reveiw_count = re.compile(r'reviewCount_')
    restaurant_reveiw_count = (yelp_restaurant
    >>find_all('span', attrs = {'class': find_reveiw_count})
    >>map(get_text)
    )
    restaurant_reveiw_count=restaurant_reveiw_count[1:]
    
    find_category = re.compile(r'priceCategory')
    restaurant_category = (yelp_restaurant
    >>find_all('div', attrs = {'class': find_category})
    >>map(get_text)                  
    )
    restaurant_category = restaurant_category[1:]
    category_filter = [line.replace('$',"") for line in restaurant_category]
    category_semi = [line.replace(',',";") for line in category_filter]
    
    joinlines = [",".join(lines) for lines in page_one]
    
    output = list(zip(final_rank_name,review_ranking,restaurant_address,restaurant_reveiw_count,category_semi))
    joinlines = [",".join(lines) for lines in output]
    joinlines_work = "\n".join(joinlines)
    return(joinlines_work)

In [31]:
page_one = webscraping_yelp('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=0')
page_one

'1. The Waterfront Restaurant & Tavern,4.5 star rating,328 Front St S,226,American (New); Seafood; Steakhouses\n2. Restore Public House,4.5 star rating,1810 State St,17,American (Traditional)\n3. Buzzard Billy’s,4 star rating,222 Pearl St,276,American (Traditional); Cajun/Creole\n4. Lovechild Restaurant,4.5 star rating,300 3rd St S,109,American (New)\n5. The Charmant,4.5 star rating,101 State St,149,French; Cocktail Bars\n6. The Freighthouse Restaurant,4 star rating,107 Vine St,125,Seafood; Steakhouses; Desserts\n7. Howie’s on La Crosse,4 star rating,1128 La Crosse St,57,American (New); Pubs\n8. Piggy’s Restaurant,4 star rating,501 Front St S,131,Steakhouses; Seafood; Sandwiches\n9. The Crow,3.5 star rating,100 3rd St S,154,American (Traditional); Gastropubs\n10. Digger’s Sting Restaurant,4 star rating,122 3rd St N,62,American (New); Steakhouses'

In [32]:
joinlines = [",".join(lines) for lines in page_one]
joinlines

['1',
 '.',
 ' ',
 'T',
 'h',
 'e',
 ' ',
 'W',
 'a',
 't',
 'e',
 'r',
 'f',
 'r',
 'o',
 'n',
 't',
 ' ',
 'R',
 'e',
 's',
 't',
 'a',
 'u',
 'r',
 'a',
 'n',
 't',
 ' ',
 '&',
 ' ',
 'T',
 'a',
 'v',
 'e',
 'r',
 'n',
 ',',
 '4',
 '.',
 '5',
 ' ',
 's',
 't',
 'a',
 'r',
 ' ',
 'r',
 'a',
 't',
 'i',
 'n',
 'g',
 ',',
 '3',
 '2',
 '8',
 ' ',
 'F',
 'r',
 'o',
 'n',
 't',
 ' ',
 'S',
 't',
 ' ',
 'S',
 ',',
 '2',
 '2',
 '6',
 ',',
 'A',
 'm',
 'e',
 'r',
 'i',
 'c',
 'a',
 'n',
 ' ',
 '(',
 'N',
 'e',
 'w',
 ')',
 ';',
 ' ',
 'S',
 'e',
 'a',
 'f',
 'o',
 'o',
 'd',
 ';',
 ' ',
 'S',
 't',
 'e',
 'a',
 'k',
 'h',
 'o',
 'u',
 's',
 'e',
 's',
 '\n',
 '2',
 '.',
 ' ',
 'R',
 'e',
 's',
 't',
 'o',
 'r',
 'e',
 ' ',
 'P',
 'u',
 'b',
 'l',
 'i',
 'c',
 ' ',
 'H',
 'o',
 'u',
 's',
 'e',
 ',',
 '4',
 '.',
 '5',
 ' ',
 's',
 't',
 'a',
 'r',
 ' ',
 'r',
 'a',
 't',
 'i',
 'n',
 'g',
 ',',
 '1',
 '8',
 '1',
 '0',
 ' ',
 'S',
 't',
 'a',
 't',
 'e',
 ' ',
 'S',
 't',
 ',',
 '1',
 '7',
 ',

In [33]:
joinlines_work = "\n".join(joinlines)
joinlines_work

'1\n.\n \nT\nh\ne\n \nW\na\nt\ne\nr\nf\nr\no\nn\nt\n \nR\ne\ns\nt\na\nu\nr\na\nn\nt\n \n&\n \nT\na\nv\ne\nr\nn\n,\n4\n.\n5\n \ns\nt\na\nr\n \nr\na\nt\ni\nn\ng\n,\n3\n2\n8\n \nF\nr\no\nn\nt\n \nS\nt\n \nS\n,\n2\n2\n6\n,\nA\nm\ne\nr\ni\nc\na\nn\n \n(\nN\ne\nw\n)\n;\n \nS\ne\na\nf\no\no\nd\n;\n \nS\nt\ne\na\nk\nh\no\nu\ns\ne\ns\n\n\n2\n.\n \nR\ne\ns\nt\no\nr\ne\n \nP\nu\nb\nl\ni\nc\n \nH\no\nu\ns\ne\n,\n4\n.\n5\n \ns\nt\na\nr\n \nr\na\nt\ni\nn\ng\n,\n1\n8\n1\n0\n \nS\nt\na\nt\ne\n \nS\nt\n,\n1\n7\n,\nA\nm\ne\nr\ni\nc\na\nn\n \n(\nT\nr\na\nd\ni\nt\ni\no\nn\na\nl\n)\n\n\n3\n.\n \nB\nu\nz\nz\na\nr\nd\n \nB\ni\nl\nl\ny\n’\ns\n,\n4\n \ns\nt\na\nr\n \nr\na\nt\ni\nn\ng\n,\n2\n2\n2\n \nP\ne\na\nr\nl\n \nS\nt\n,\n2\n7\n6\n,\nA\nm\ne\nr\ni\nc\na\nn\n \n(\nT\nr\na\nd\ni\nt\ni\no\nn\na\nl\n)\n;\n \nC\na\nj\nu\nn\n/\nC\nr\ne\no\nl\ne\n\n\n4\n.\n \nL\no\nv\ne\nc\nh\ni\nl\nd\n \nR\ne\ns\nt\na\nu\nr\na\nn\nt\n,\n4\n.\n5\n \ns\nt\na\nr\n \nr\na\nt\ni\nn\ng\n,\n3\n0\n0\n \n3\nr\nd\n \nS\nt\n \nS\n,\n1\n0\n

## <font color="red">  Problem 5 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

In [34]:
webpage_number = re.compile(r'pagination-link')
page_num = (yelp_restaurant
>>find_all('a', attrs = {'class': webpage_number})
>>map(tlz.get('href'))
    )
page_num

['/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=10',
 '/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=20',
 '/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=30',
 '/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=40',
 '/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=50',
 '/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=60',
 '/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=70',
 '/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=80']

In [35]:
add_http = [link.replace("/","https://wwww.yelp.com/") for link in page_num]
add_http

['https://wwww.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=10',
 'https://wwww.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=20',
 'https://wwww.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=30',
 'https://wwww.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=40',
 'https://wwww.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=50',
 'https://wwww.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=60',
 'https://wwww.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=70',
 'https://wwww.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=80']

In [36]:
restof_pages = [webscraping_yelp(pages) for pages in add_http]
restof_pages

['11. Schuby’s Neighborhood Butcher,4.5 star rating,321 State St,17,Butcher; Delis; Caterers\n12. Uno Venti Pizzeria,4 star rating,120 King St,17,Pizza; Italian; Beer Bar\n13. 4 Sisters Wine Bar and Tapas,3.5 star rating,100 Harborview Plz,111,Wine Bars; Tapas/Small Plates; Tapas Bars\n14. Milwaukee Burger Company,3.5 star rating,3039 Medco Ct,23,Burgers; Sports Bars; Beer Bar\n15. Five Star Eggrolls,3.5 star rating,1203 La Crosse St,9,Thai; Laotian; Chinese\n16. River Rats Bar and Grill,4.5 star rating,1311 La Crescent Pl,28,American (New); Burgers; Cocktail Bars\n17. Hmong’s Golden Egg Roll,4 star rating,901 State St,69,Laotian; Vietnamese; Thai\n18. Señor Villa,4 star rating,325 West Ave N,69,Mexican\n19. Burritos House,4.5 star rating,1205 La Crosse St,30,Mexican\n20. Le Chateau,4.5 star rating,410 Cass St,51,French; Wine Bars; Cocktail Bars',
 '21. Iguana’s Mexican Street Café,4 star rating,1800 State St,71,Mexican\n22. Hungry Peddler,4 star rating,3429 Mormon Coulee Rd,64,America

In [37]:
restof_pages_work = '\n'.join(restof_pages)
restof_pages_work

'11. Schuby’s Neighborhood Butcher,4.5 star rating,321 State St,17,Butcher; Delis; Caterers\n12. Uno Venti Pizzeria,4 star rating,120 King St,17,Pizza; Italian; Beer Bar\n13. 4 Sisters Wine Bar and Tapas,3.5 star rating,100 Harborview Plz,111,Wine Bars; Tapas/Small Plates; Tapas Bars\n14. Milwaukee Burger Company,3.5 star rating,3039 Medco Ct,23,Burgers; Sports Bars; Beer Bar\n15. Five Star Eggrolls,3.5 star rating,1203 La Crosse St,9,Thai; Laotian; Chinese\n16. River Rats Bar and Grill,4.5 star rating,1311 La Crescent Pl,28,American (New); Burgers; Cocktail Bars\n17. Hmong’s Golden Egg Roll,4 star rating,901 State St,69,Laotian; Vietnamese; Thai\n18. Señor Villa,4 star rating,325 West Ave N,69,Mexican\n19. Burritos House,4.5 star rating,1205 La Crosse St,30,Mexican\n20. Le Chateau,4.5 star rating,410 Cass St,51,French; Wine Bars; Cocktail Bars\n21. Iguana’s Mexican Street Café,4 star rating,1800 State St,71,Mexican\n22. Hungry Peddler,4 star rating,3429 Mormon Coulee Rd,64,American (T

In [38]:
csv_content_one = joinlines_work

In [39]:
with open('Yelp_LaCrosse_Restaurants_final.csv','w') as outfile:
    outfile.write(csv_content_one)

In [40]:
with open('Yelp_LaCrosse_Restaurants_final.csv','a') as outfile:
    outfile.write(restof_pages_work)

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.